### Calculate the density of tourist attractions in each LSOA in London 

In [11]:
!pip install --upgrade osmnx

In [12]:
import osmnx as ox
print(ox.__version__)  # Should be 1.2.2 or newer

2.0.0


In [17]:
import geopandas as gpd
import osmnx as ox
import pandas as pd

# Step 1: Load London LSOA boundary data
lsoa_boundaries = gpd.read_file("data/London/LSOA_2011_London_gen_MHW.shp")

# Step 2: Retrieve tourist attraction data for London
# Use OSMnx to extract nodes tagged as `tourism=attraction`
tourist_attractions = ox.features_from_place(
    "London, England", tags={"tourism": "attraction"}
)

# Ensure the coordinate reference system (CRS) is consistent
tourist_attractions = tourist_attractions.to_crs(lsoa_boundaries.crs)

# Step 3: Count the number of tourist attractions within each LSOA
lsoa_boundaries["tourist_attraction_count"] = lsoa_boundaries.apply(
    lambda row: tourist_attractions.within(row.geometry).sum(), axis=1
)

# Step 4: Calculate the density of tourist attractions (per square kilometer)
# Transform to an equal-area projection to compute accurate area
lsoa_boundaries = lsoa_boundaries.to_crs({"proj": "cea"})
lsoa_boundaries["area_km2"] = lsoa_boundaries.geometry.area / 1e6  # Convert to square kilometers

# Tourist attraction density = count / area
lsoa_boundaries["tourist_attraction_density"] = (
    lsoa_boundaries["tourist_attraction_count"] / lsoa_boundaries["area_km2"]
)

# Step 5: Export results to a CSV file
# Use 'LSOA11CD' as the LSOA code column
output_path = "data/london_lsoa_tourist_density.csv"
output_csv = lsoa_boundaries[["LSOA11CD", "tourist_attraction_count", "tourist_attraction_density"]]
output_csv.to_csv(output_path, index=False)

print(f"Tourist attraction density has been saved to {output_path}")

Tourist attraction density has been saved to data/london_lsoa_tourist_density.csv
